In [6]:
nsfw_words = []

with open("nsfw.txt", "r") as f:
    nsfw_words = f.read().split("\n")

In [7]:
import pandas as pd
import pyarrow.parquet as pq
import glob
from tqdm import tqdm
tqdm.pandas()

def dataframe(path, cols):
	paths = glob.glob(path)
	dataset = pq.ParquetDataset(paths)
	table = dataset.read()
	df = table.to_pandas()
	df = df[cols]
	return df

In [8]:
df = dataframe('output/*/*.parquet', ['word', 'definition', 'score'])

In [22]:
def detect_nsfw(row):
    d = row.definition.split()
    for dw in d:
        for nw in nsfw_words:
            if (nw.lower() in dw.lower()): return True
    
    return False

df['nsfw'] = df.progress_apply(lambda row: detect_nsfw(row), axis=1)

100%|██████████| 466111/466111 [02:27<00:00, 3160.59it/s]


In [27]:
df[df['nsfw'] == True].shape

(147885, 4)

In [28]:
df = df.drop(df[df['nsfw'] == True].index)

In [33]:
def has_number(row):
    for i in range(10):
        if (str(i) in row.word): return True
    
    return False

df['has_number'] = df.progress_apply(lambda row: has_number(row), axis=1)

100%|██████████| 318226/318226 [01:05<00:00, 4860.86it/s]


In [35]:
df[df['has_number'] == True].shape

(3798, 6)

In [36]:
df = df.drop(df[df['has_number'] == True].index)

In [37]:
def format_definition(row):
    definition = row.definition.replace("[", "")
    definition = definition.replace("]", "")
    definition = definition.replace(";;", "/")
    return definition

df['definition'] = df.progress_apply(lambda row: format_definition(row), axis=1)

100%|██████████| 314428/314428 [00:08<00:00, 36628.98it/s]


In [38]:
df['definition'][:10]

0         verb: to own, especially in a said situation.
1     The combination of an 11 incher and the extrem...
2                         To be close to, near, almost.
3                 One who gives a eugoogly at a funeral
4     an object or a person that is Lame, Whack, or ...
5           Correspondence received while incarcerated.
6     When someone is taking hecka long. or when som...
8     Its both a hug and cuddle combinded into one m...
9     A very lucky, beautiful, godess like person. W...
10    adjective - To be incapacitated due to a varie...
Name: definition, dtype: object

In [39]:
df['pat'] = df['score'].apply(lambda s: round(s, 1))

In [40]:
df.to_parquet('output', engine='pyarrow', partition_cols = ['pat'])